In [1]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.image import Image
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice


In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

demoaml
demo-aml-use
eastus
36cfc6d6-79ca-4642-b263-93d6eaa4a823


In [3]:
akscompute_cluster_name = "k8cpucluster"
image_name = "diabclassprob"
service_name = "diabdemo2"

In [4]:
found = False

# Check if this compute target already exists in the workspace.

cts = ws.compute_targets

if akscompute_cluster_name in cts and cts[akscompute_cluster_name].type == 'AKS':
    found = True
    print('Found existing compute target.')
    deployment_target = cts[akscompute_cluster_name]

if not found:
    print('Creating a new compute target...')
        
    prov_config = AksCompute.provisioning_configuration()
    prov_config.enable_ssl(leaf_domain_label = service_name)
    
    # Create the cluster
    deployment_target = ComputeTarget.create(workspace = ws, 
                                      name = akscompute_cluster_name, 
                                      provisioning_configuration = prov_config)

    deployment_target.wait_for_completion(show_output = True)

    print(deployment_target.provisioning_state)
    print(deployment_target.provisioning_errors)

Found existing compute target.


In [5]:
image_list = Image.list(workspace = ws, image_name = image_name)

In [6]:
image = image_list[0]
print("Name: ", image.name, ",Version: ", image.version)

Name:  diabclassprob ,Version:  6


In [7]:
aks_config = AksWebservice.deploy_configuration(num_replicas=2)

In [8]:
%%time
aks_service_name = service_name

aks_service = Webservice.deploy_from_image(workspace = ws, 
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = deployment_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Creating service
Running.
Failed

Service deployment polling reached non-successful terminal state, current service state: Failed
Error:
{
  "code": "KubernetesError",
  "statusCode": 400,
  "message": "Kubernetes Deployment Error",
  "details": [
    {
      "code": "Unschedulable",
      "message": "0/3 nodes are available: 3 Insufficient cpu."
    }
  ]
}

Service deployment polling reached non-successful terminal state, current service state: Failed
Error:
{
  "code": "KubernetesError",
  "statusCode": 400,
  "message": "Kubernetes Deployment Error",
  "details": [
    {
      "code": "Unschedulable",
      "message": "0/3 nodes are available: 3 Insufficient cpu."
    }
  ]
}



WebserviceException: Service deployment polling reached non-successful terminal state, current service state: Failed
Error:
{
  "code": "KubernetesError",
  "statusCode": 400,
  "message": "Kubernetes Deployment Error",
  "details": [
    {
      "code": "Unschedulable",
      "message": "0/3 nodes are available: 3 Insufficient cpu."
    }
  ]
}

In [13]:
aks_service.get_logs()

"2019-06-24T16:18:24,186719293+00:00 - rsyslog/run \n2019-06-24T16:18:24,187349694+00:00 - iot-server/run \n2019-06-24T16:18:24,188339897+00:00 - gunicorn/run \n2019-06-24T16:18:24,207161644+00:00 - nginx/run \nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2019-06-24T16:18:24,407070348+00:00 - iot-server/finish 1 0\n2019-06-24T16:18:24,416855173+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 19.6.0\nListening at: http://127.0.0.1:9090 (12)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 47\n'pattern' package not found; tag filters are not available for English\nInitializing logger\nStarting up app insights client\nStarting up request id generator\nStarting up app insight hooks\nInvoking user's init function\n2019-06-24 16:18:31,540 | azureml.core.run | DEBUG | Could not load run context Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an 

In [14]:
print("ScoringUri: ", aks_service.scoring_uri)
print("Swagger Uri: ", aks_service.swagger_uri)
key1,key2 = aks_service.get_keys()

ScoringUri:  https://diabdemo1h8qz2b.eastus.cloudapp.azure.com:443/api/v1/service/diabdemo1/score
Swagger Uri:  https://diabdemo1h8qz2b.eastus.cloudapp.azure.com:443/api/v1/service/diabdemo1/swagger.json


In [15]:
# construct raw HTTP request and send to the service
import requests
import json

test_sample = json.dumps({'data': [
    	{
					"pregnancies": "6",
	    			"plasma glucose": "148",
	    			"blood pressure": "72",
	    			"triceps skin thickness": "35",
	    			"insulin": "0",
	    			"bmi": "33.6",
	    			"diabetes pedigree": "0.627",
	    			"age": "50"
				},
				{
					"pregnancies": "1",
	    			"plasma glucose": "85",
	    			"blood pressure": "66",
	    			"triceps skin thickness": "29",
	    			"insulin": "0",
	    			"bmi": "26.6",
	    			"diabetes pedigree": "0.351",
	    			"age": "31"
				}
]})
test_sample = bytes(test_sample,encoding = 'utf8')

# Don't forget to add key to the HTTP header.
headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + key1}

resp = requests.post(aks_service.scoring_uri, test_sample, headers=headers)


print("prediction:", resp.text)


prediction: [{"prediction": 1.0, "probability": 0.5834204702}, {"prediction": 0.0, "probability": 0.8421741844}]


In [16]:
print(key1)

al7ovKSd3CmEkveuGqWWbSXw6sNKvy0x
